In [ ]:
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib
import rasterio

from glob import glob

import fct_misc
from fct_stats import pca_procedure

In [ ]:
DATA_PATH='/mnt/data-01/gsalamin/deperissement-hetre/'

NORTH_ORTHO=os.path.join(DATA_PATH, 'initial/True_ortho/Tiled_North/')
SOUTH_ORTHO=os.path.join(DATA_PATH, 'initial/True_ortho/Tiled_South/')

NORTH_MASK=os.path.join(DATA_PATH, 'processed/CHM/CHM_binary_beurnevesin.tif')
SOUTH_MASK=os.path.join(DATA_PATH, 'processed/CHM/CHM_binary_miecourt.tif')

AOI_GT=os.path.join(DATA_PATH, 'initial/AOI/AOI_tiles/AOI_GT.shp')

BEECHES_POLYGONS=os.path.join(DATA_PATH, 'processed/trees/ok_trees.gpkg')
BEECHES_LAYER='ok_trees_20_40_m'

written_files=[]


# Import data

In [ ]:
beeches=gpd.read_file(BEECHES_POLYGONS, layer=BEECHES_LAYER)

In [ ]:
tiles_list_north=glob(NORTH_ORTHO)
tiles_list_south=glob(SOUTH_ORTHO)

In [ ]:
north_chm=fct_misc.polygonize_binary_raster(NORTH_MASK)
south_chm=fct_misc.polygonize_binary_raster(SOUTH_MASK)

chm=pd.concat([north_chm, south_chm])
del north_chm, south_chm


In [ ]:
aoi_gt=gpd.read_file(AOI_GT)

In [ ]:
aoi_gt.head()

Treat pixel values

In [ ]:
fct_misc.test_crs(beeches.crs, chm.crs)
correct_high_beeches=gpd.overlay(beeches, chm)
correct_high_beeches.drop(columns=['class'], inplace=True)

In [ ]:
tiles_south=aoi_gt[aoi_gt['NAME'].str.startswith('258')]
beeches_south=correct_high_beeches[correct_high_beeches['zone']=='Miecourt']
beeches_on_tiles_south=gpd.overlay(beeches_south[['no_arbre', 'etat_sanitaire', 'geometry']],
                                    tiles_south[['NAME', 'geometry']])
beeches_on_tiles_south['filepath']=[os.path.join(SOUTH_ORTHO, 'South_ortho_JUHE_LV95_NF02_3cm_' + name + '.tif')
                                        for name in beeches_on_tiles_south['NAME'].values]


In [ ]:
del tiles_south, beeches_south

In [ ]:
pixels_south=pd.DataFrame()
for beech in beeches_on_tiles_south.itertuples():
    pixels=fct_misc.get_pixel_values(beech.geometry, beech.filepath, bands=range(1,5),
                                    health_status=beech.etat_sanitaire)

    pixels_south=pd.concat([pixels_south, pixels])

In [ ]:
tiles_north=aoi_gt[aoi_gt['NAME'].str.startswith('257')]
beeches_north=correct_high_beeches[correct_high_beeches['zone'].str.startswith('Beurnevesi')]
beeches_on_tiles_north=gpd.overlay(beeches_north[['no_arbre', 'etat_sanitaire', 'geometry']],
                                    tiles_north[['NAME', 'geometry']])
beeches_on_tiles_north['filepath']=[os.path.join(NORTH_ORTHO, 'North_ortho_JUHE_LV95_NF02_3cm_' + name + '.tif')
                                        for name in beeches_on_tiles_north['NAME'].values]

In [ ]:
del tiles_north, beeches_north

In [ ]:
pixels_north=pd.DataFrame()
for beech in beeches_on_tiles_north.itertuples():
    pixels=fct_misc.get_pixel_values(beech.geometry, beech.filepath, bands=range(1,5),
        health_status=beech.etat_sanitaire)

    pixels_north=pd.concat([pixels_north, pixels])

In [ ]:
pixels_beeches=pd.concat([pixels_north, pixels_south], ignore_index=True)

In [ ]:
del pixels_north, pixels_south, pixels

In [ ]:
pixels_beeches.rename(columns={'band1':'Rouge', 'band2':'Vert', 'band3':'Bleu', 'band4':'Proche IR'}, inplace=True)


In [ ]:
pixels_beeches['NDVI']=(pixels_beeches['Proche IR'].astype('float64')-
                            pixels_beeches['Rouge'].astype('float64'))/(pixels_beeches['Proche IR'].astype('float64')+
                                                                            pixels_beeches['Rouge'].astype('float64'))

pixels_beeches.loc[pixels_beeches['health_status']=='sain', 'health_status']='1. sain'
pixels_beeches.loc[pixels_beeches['health_status']=='malade', 'health_status']='2. malade'
pixels_beeches.loc[pixels_beeches['health_status']=='mort', 'health_status']='3. mort'


In [ ]:
pixels_beeches[pixels_beeches['NDVI']>=0.90]  # .to_csv('test.csv') 

In [ ]:
pixels_beeches=pixels_beeches[pixels_beeches['NDVI']<0.90]

In [ ]:
table_path=fct_misc.ensure_dir_exists(os.path.join(DATA_PATH, 'final/tables'))
im_path=fct_misc.ensure_dir_exists(os.path.join(DATA_PATH, 'final/images'))

In [ ]:
boxplots=pixels_beeches.plot.box(by='health_status',
                            title='Distribution des pixels en fonction de l\'état sanitaire',
                            figsize=(16,5),
                            grid=True)
fig = boxplots[0].get_figure()
filename=os.path.join(im_path, 'bxplt_distribution_status_health.jpg')
fig.savefig(filename, bbox_inches='tight')
written_files.append(filename)

In [ ]:
features = pixels_beeches.columns.tolist()
features.remove('health_status')
to_describe='health_status'

written_files_pca_pixels=pca_procedure(pixels_beeches, features, to_describe,
                        table_path, im_path, 
                        file_prefix=f'PCA_beeches_',
                        title_graph='PCA for the values of the pixels on each band and the NDVI')

written_files.extend(written_files_pca_pixels)

Calculate NDVI rasters

In [ ]:
tiles_list=[]
tiles_list.extend(tiles_list_north)
tiles_list.extend(tiles_list_south)

In [ ]:
for tile in tiles_list:
    with rasterio.open(tile) as src:
        image = src.read()
        im_profile=src.profile

    red_band=image[0].astype('float32')
    nir_band=image[3].astype('float32')
    ndvi_tile=np.divide((nir_band - red_band),(nir_band + red_band),
                        out=np.zeros_like(nir_band - red_band),
                        where=(nir_band + red_band)!=0)

    im_profile.update(count= 1,
                    dtype='float32')
    tile_path=os.path.join(DATA_PATH, 'processed/NDVI', tile.split('/')[-1])
    with rasterio.open(tile_path.replace('ortho_JUHE_LV95_NF02_3cm', 'NDVI'), 'w', **im_profile) as dst:
        dst.write(ndvi_tile,1)
        written_files.append(tile_path)

In [ ]:
print('Some files were written:')
for file in written_files:
    print(file)